In [4]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
import apache_beam.runners.interactive.interactive_beam as ib

In [5]:
runner = interactive_runner.InteractiveRunner()

In [7]:
p = beam.Pipeline(runner = runner)

nums = p | "Create numbers" >> beam.Create(range(1,11))
squares = nums | "Squere" >> beam.Map(lambda x: x*X0)
